# 2023-12-01 What is uncertainty (also called error or precision) in science?

## Overview

Any scientific result should be presented as a measured value and estimated uncertainty in that value. There are multiple ways to think about the uncertainty, depending on the circumstances. 

It is almost never correct to think about it as an "error" -- that implies it is something you can fix if you just try harder. If that is the case you should really just try harder before reporting the result.

There are two fairly common cases that are useful to think about:

+ You are measuring a process that is truly random. No matter how good your equipment is you will measure a diffferent value every time. In this case it is useful to think about the uncertainty as expressing the range of values you should expect to see if you repeat the experiment. An example of this is measuring the flux from a star: it is a random variable and you would measure a different value each time even if circumstances were identical.
+ There is a limit on the precision of what you can measure. In this case, future measurements might be better than what you can do now. In this case it is useful to think about the uncertainty as expression the range of values consistent with the precision with which you can measure that thing. An example of this from astronomy is time: we can measure time much more precisely now than we could 100 years ago.

### One goal for today: get a sense of uncertainty in measuring exoplanet properties

## Run the wall of code below -- it is a transit measurement for TIC 402828941

In [ ]:
%matplotlib widget

from astropy.table import Table 
from astropy.time import Time
from astropy.timeseries import TimeSeries, aggregate_downsample
from astropy import units as u

import ipywidgets as ipw
import mpl_interactions.ipyplot as iplt
import matplotlib.pyplot as plt
import numpy as np
import batman


In [ ]:
tab = Table.read('TIC-402828941-2022-08-04-transformed-relative-flux.csv')
tic = tab[tab['star_id'] == 1]
t_mean = tic['BJD'].mean()

t_ob = Time(tic['BJD'], scale='tdb', format='jd')

In [ ]:
bad_beginning = t_ob < t_ob[0] + 15 * u.min

tic = tic[~bad_beginning]
t_ob = t_ob[~bad_beginning]

after_transit = t_ob.jd > 2459795.85

norm_fac = 1 / tic['relative_flux'][after_transit].mean()


tic['relative_flux'] *= norm_fac

In [ ]:

ts = TimeSeries([tic['relative_flux']], time=t_ob)

bts = aggregate_downsample(ts, time_bin_size=10 * u.min)

In [ ]:
params = batman.TransitParams()

params.t0 = t_mean                        #time of inferior conjunction
params.per = 1.                       #orbital period
params.rp = 0.1                       #planet radius (in units of stellar radii)
params.a = 15.                        #semi-major axis (in units of stellar radii)
params.inc = 87.                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 90.                        #longitude of periastron (in degrees)
params.limb_dark = "quadratic"        #limb darkening model
params.u = [0.1, 0.1]      #limb darkening coefficients [u1, u2, u3, u4]

t = t_mean + np.linspace(-0.2, 0.2, 1000)  #times at which to calculate light curve
m = batman.TransitModel(params, t)    #initializes model

In [ ]:
def make_set_vals(controls, vals):
    def set_vals(change):
        rp = controls.controls['planet_radius'].children[1].value
        semi_major = controls.controls['semi_major_axis'].children[1].value
        t0_shift = controls.controls['t0_shift'].children[1].value
        depth = 1 - m.light_curve(params).min()
        duration = t[m.light_curve(params) < 1]
        duration = (duration[-1] - duration[0]) * 24
        vals.value = (f'<b>R_p/R_*</b> = {rp}</br>'
                      f'<b>a/R_*</b> = {semi_major}</br>'
                      f'<b>depth</b> = {depth:.4f} ({1000 * depth:.1f} ppt)({1e6 * depth:.0f} ppm)</br>'
                      f'<b>duration</b> = {duration:.2f} hours</br>'
                      f'<b>T_c</b> = {t_mean + float(t0_shift):.5f}')
        
    return set_vals

In [ ]:
def f3(t, planet_radius, semi_major_axis, t0_shift):
    params.rp = planet_radius
    params.a = semi_major_axis
    params.t0 = t_mean + t0_shift
    return m.light_curve(params)


def f_data(t):
    return tic['relative_flux']

def f_bin_data(t):
    return bts['relative_flux']

fig, ax = plt.subplots()
controls = iplt.plot(t, f3, 
                     planet_radius=np.linspace(0.01, 0.2, 100), 
                     semi_major_axis=np.linspace(1.5, 10, 100), 
                     t0_shift=np.linspace(-0.1, 0.1, 100),
                     label="Transit model")

vals = ipw.HTML(value='Transit parameters')

set_v = make_set_vals(controls, vals)

for control in controls.controls.values():
    label = control.children[1]
    label.observe(set_v, names='value')
    
_ = iplt.plot(tic['BJD'], f_data, '.', color='red', alpha=0.3, label='all data')

_ = iplt.plot(bts.time_bin_center.jd, f_bin_data, '.', color='green', label='binned data')

_ = plt.legend()
plt.xlabel('Time (BJD_TDB)')
plt.ylabel('Relative flux')
plt.grid()
vals

## What to do


### Interactive graph

+ Adjust the sliders until the blue curve, which represents a mathematical model of a transit, matches the data points reasonably well
+ Write the values you get for the parameters below on the clipboard at the front of the class:
    - R_p/R_*
    - a/R_*
    - depth
    - duration
    - T_C 
+ Find at least one additional set of parameters (slider values) that you think fits the data about as well as the first set you chose.

### Compare your parameters with the "official" values

+ You can look up the parameters by going to this site: https://exofop.ipac.caltech.edu/tess/target.php?id=402828941
